In [1]:
from accelerate import Accelerator
import torch

accelerator = Accelerator()
accelerator.state.num_processes = 3  # For 3 GPUs
device = accelerator.device
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [2]:
import os,sys, json,re,pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn

from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words

In [3]:
bin_path = '/home/raisul/DATA/temp/x86_pe_msvc_O2_static'
bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
ground_truth_path ='/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4/'  
MODEL_SAVE_PATH= '/home/raisul/probabilistic_disassembly/models/'
EXPERIMENT_NAME = 'prototype_elf_asis'
MAX_TOKEN_SIZE = 120
MAX_SEQUENCE_LENGTH = 10
VOCAB_SIZE = 500
BATCH_SIZE = 600

In [4]:
def replace_num_with_word(input_string , replace_dict):
    def num_to_word(match):
        number = int( match.group(0))
        return num2words(replace_dict[number]).replace(' ','').replace('-',"")
    result_string = re.sub(r'\b\d+\b', num_to_word, input_string)
    return result_string



def replace_hex_with_decimal(input_string):
    # Regular expression to find hexadecimal numbers prefixed with "0x" or "0X"
    hex_pattern = r'0[xX][0-9a-fA-F]+'
    
    # Function to convert each found hex number to decimal
    def hex_to_decimal(match):
        hex_value = match.group(0)  # Extract the matched hex number
        decimal_value = str(int(hex_value, 16))  # Convert hex to decimal
        return decimal_value
    # Substitute all hex numbers in the string with their decimal equivalents
    result_string = re.sub(hex_pattern, hex_to_decimal, input_string)
    return result_string



In [5]:

def get_ground_truth_ghidra(exe_path, text_section_offset , text_section_len):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    ghidra_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(ghidra_file_path, "r") as file:
        ghidra_data = json.load(file)

    ground_truth_offsets = list(ghidra_data.keys())

    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()
    return ground_truth_offsets



def find_data_in_textsection(ground_truth_offsets , text_section_offset , text_section_len, offset_inst_dict):
    data_offsets = []
    for i in range(1, len(ground_truth_offsets)-1):
        distance = ground_truth_offsets[i+1] - ground_truth_offsets[i]

        inst_len = offset_inst_dict[ground_truth_offsets[i]].size 
        
        if distance!=inst_len:
            # print('offset_ranges[i]: ',ground_truth_offsets[i] , 'offset_ranges[i-1]: ',ground_truth_offsets[i-1], ' inst_len: ',inst_len  )
            
            # print(ground_truth_offsets[i],' ' ,hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]], ' len',offset_inst_dict[ground_truth_offsets[i]].size )
            # print("\nByte GAP ###### ",distance ,' Missing bytes: ', distance - inst_len)
            
            for j in range( ground_truth_offsets[i] +inst_len , ground_truth_offsets[i+1]  ):
                data_offsets.append(j)
                # if offset_inst_dict[j]:
                #     print("# ",j, offset_inst_dict[j].mnemonic, offset_inst_dict[j].op_str , 'inst len:',offset_inst_dict[j].size )
                # else:
                #     print("# ",j, " invalid ")
            # print('\n')
        else:
            # print(ground_truth_offsets[i],' ', hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]].mnemonic,offset_inst_dict[ground_truth_offsets[i]].op_str ,' len',offset_inst_dict[ground_truth_offsets[i]].size)
            pass
    return data_offsets
            

In [6]:





def linear_sweep(offset_inst , target_offset):
    inst_sequence = ''
    address_list = []
    
    current_offset = target_offset
    for q in range(MAX_SEQUENCE_LENGTH):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:
                return  None
                
            current_offset = current_offset + current_instruction.size
            inst_sequence+= str( hex(current_instruction.address)) +" "+ current_instruction.mnemonic +' '+ current_instruction.op_str+ ' ; ' 
            address_list.append(current_instruction.address)
            
            
            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                break
                

    return inst_sequence, address_list


SEQUENCES = []
LABELS     = []

for bin_file_path in bin_files:

    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    
    with open(bin_file_path, 'rb') as f:

        try:
            elffile = ELFFile(f)
           
            textSection = elffile.get_section_by_name('.text').data()
        
            text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
          
            ground_truth_offsets = get_ground_truth_ghidra(bin_file_path, text_section_offset , len(textSection))
            
        except Exception as e:
            print("An error occurred:", e ,bin_file_path)
            continue

    for byte_index in range(len(textSection)):
        try:    

            instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
            offset_inst[text_section_offset+byte_index] = instruction
            
            # if instruction:
            #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
            # else:
            #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
                
            

        except Exception as e:
            print(traceback.print_exc() )
            print(e)

    
    
    offset_inst_dict = collections.OrderedDict(sorted(offset_inst.items()))

    DATA_OFFSETS = find_data_in_textsection(ground_truth_offsets , text_section_offset , len(textSection) , offset_inst)


    
    for byte_offset in range(text_section_offset, text_section_offset+len(textSection)):
        return_value = linear_sweep(offset_inst_dict ,  byte_offset )
        if return_value== None:
            continue
        inst_seq, inst_addresses = return_value 
        ###################################################################
        ## number to words
        disassembly_decimal = replace_hex_with_decimal(inst_seq)

        #num to words all
        numbers = [int(s) for s in re.findall(r'\b\d+\b', disassembly_decimal)]
        numbers = sorted(set(numbers) , reverse=True)
        number_word_dict = {}
        
        for ix,n in enumerate(numbers):
            number_word_dict[n] = len(numbers)-1 -ix

        disassembly_num_to_words = replace_num_with_word(disassembly_decimal , number_word_dict)

        


        
        ###########################################################################
        
        
        SEQUENCES.append(disassembly_num_to_words)
        if byte_offset in ground_truth_offsets:
            LABELS.append(float(1))
        else:
            LABELS.append(float(0))



An error occurred: Magic number does not match /home/raisul/DATA/x86_O2_d4/T1C641EF11DCF618412177243AAF8B2428753AA427259EEF10B8EE1AF0EF941B45DFCB94
An error occurred: Magic number does not match /home/raisul/DATA/x86_O2_d4/T191032169836D22C2F3A79125B26074907F9EE221B075542DFC5C53886BCB2D48F72F7A
An error occurred: Magic number does not match /home/raisul/DATA/x86_O2_d4/T1E3A28485361962326DB15B6BCF1F6098E72AC033121D965431CDC2B16F7287A933BFE9
An error occurred: [Errno 2] No such file or directory: '/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4/T17201A2ED29F75581F037B629069A646A3138D4932CC8BF64BD2E01138F06DAC477D791.json' /home/raisul/DATA/x86_O2_d4/T17201A2ED29F75581F037B629069A646A3138D4932CC8BF64BD2E01138F06DAC477D791
An error occurred: Magic number does not match /home/raisul/DATA/x86_O2_d4/T134E132915CEF60963173622F8B9B6425300CF81721E8FE69FD0E614CAF018A9D6F5E9C
An error occurred: Magic number does not match /home/raisul/DATA/x86_O2_d4/T1B431ED90AC6538518623C2B7962751C8C215E0671699CF92FC

In [7]:
print(len(SEQUENCES) , len(LABELS))
print(LABELS.count(0), LABELS.count(1))

391845925 391845925
274040987 117804938


In [8]:
import sys,os

from transformers import BertTokenizer,BertForSequenceClassification

# If using a character-level tokenizer for sequences like DNA/Protein:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")#BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer = tokenizer.train_new_from_iterator(SEQUENCES, VOCAB_SIZE)

tokenizer.save_pretrained(MODEL_SAVE_PATH +"bert_tokenizer" )



('/home/raisul/probabilistic_disassembly/models/bert_tokenizer/tokenizer_config.json',
 '/home/raisul/probabilistic_disassembly/models/bert_tokenizer/special_tokens_map.json',
 '/home/raisul/probabilistic_disassembly/models/bert_tokenizer/vocab.txt',
 '/home/raisul/probabilistic_disassembly/models/bert_tokenizer/added_tokens.json',
 '/home/raisul/probabilistic_disassembly/models/bert_tokenizer/tokenizer.json')

In [9]:
SEQUENCES[0]


'five endbr64  ; six sub rsp, two ; seven mov rdi, qword ptr [rsi + two] ; eight xor eax, eax ; nine call four ; ten lea rsi, [rip + three] ; eleven mov edi, zero ; twelve lea edx, [rax + rax*zero + zero] ; thirteen imul edx, eax ; fourteen sub edx, one ; '

In [10]:
class BinaryDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        # Tokenize the 
        tokenized_text = (self.tokenizer(text , max_length= MAX_TOKEN_SIZE,padding='max_length', truncation=True , return_tensors='pt')).to(device)
        
        return tokenized_text, label
        
    def __len__(self):
        return len(self.texts)

In [11]:
print('starting')
dataset = BinaryDataset(SEQUENCES, LABELS, tokenizer)
with open(MODEL_SAVE_PATH+'training_data.pkl', 'wb') as f:
    pickle.dump(dataset, f)

# Load from file
with open(MODEL_SAVE_PATH+'training_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

print(len(loaded_data))  # Output: {'name': 'Alice', 'age': 25, 'city': 'New York'}

# train_size = int(0.8 * len(dataset))
# validation_size = len(dataset) - train_size

# train_dataset  = torch.utils.data.Subset(dataset, range(train_size))
# validation_dataset = torch.utils.data.Subset(dataset, range(train_size , len(dataset)))

# # train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size] , generator=torch.Generator().manual_seed(42))

starting
391845925
